In [1]:
!sudo pip3 install --upgrade pip tensorflow keras numpy pandas pillow sklearn opencv-python optuna scikit-image  optkeras h5py==2.10.0

     |████████████████████████████████| 1.5 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 394.3 MB 39 kB/s s eta 0:00:01  |▊                               | 8.9 MB 634 kB/s eta 0:10:08     |█                               | 12.8 MB 678 kB/s eta 0:09:23     |██                              | 26.0 MB 3.7 MB/s eta 0:01:41     |██▋                             | 32.6 MB 8.2 MB/s eta 0:00:45     |███▍                            | 41.4 MB 10.3 MB/s eta 0:00:35     |███▍                            | 42.2 MB 10.3 MB/s eta 0:00:35     |███▌                            | 42.7 MB 10.3 MB/s eta 0:00:35     |████▎                           | 52.3 MB 6.0 MB/s eta 0:00:57     |█████▎                          | 65.3 MB 15.6 MB/s eta 0:00:22     |█████▊                          | 70.6 MB 24.4 MB/s eta 0:00:14     |███████                         | 87.1 MB 22.0 MB/s eta 0:00:14
     |████████████████████████████████| 14.8 MB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 9

In [2]:
# !sudo pip3 install keras==2.1.5 tensorflow==1.13.1 

In [7]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *
props = Properties()
props["pegasus.mode"] = "development"
props.write()

#Replica Catalog
rc = ReplicaCatalog()
in_files=[]

label_csv = './inputs/training_solutions_rev1.csv'

for file in glob("./inputs/*.jpg"):
    f = file.replace("./inputs/", '')
    in_files.append(File(f))
    rc.add_replica("local", File(f), Path("./inputs/").resolve() / f)
    
csv = label_csv.replace("./inputs/",'')
rc.add_replica("local", File(csv), Path("./inputs/").resolve() / csv)
rc.write()
#Transformation

data_label = Transformation( "Step1.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Step1.py",
            is_stageable=True
            )
training_preprocess1 = Transformation( "Training_Preprocess1.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Training_Preprocess1.py",
            is_stageable=True
            )
training_preprocess2 = Transformation( "Training_Preprocess2.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Training_Preprocess2.py",
            is_stageable=True
            )
val_preprocess = Transformation( "Val_Preprocess.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Val_Preprocess.py",
            is_stageable=True
            )
test_preprocess = Transformation( "Test_Preprocess.py",
            site="local",
            pfn="/home/scitech/shared-data/Galaxy2/Test_Preprocess.py",
            is_stageable=True
            )
tc = TransformationCatalog()\
    .add_transformations(data_label,training_preprocess1,training_preprocess2,val_preprocess, test_preprocess)\
    .write()

#Workflow
wf = Workflow("Galaxy", infer_dependencies=True)

dataset_without_file = []
y = []
for i in range(5):
    dataset_without_file.append('Class0_'+ str(i)+'.jpg')
    y.append(0)
    dataset_without_file.append('Class1_'+ str(i)+'.jpg')
    y.append(1)
    dataset_without_file.append('Class2_'+ str(i)+'.jpg')
    y.append(2)
    dataset_without_file.append('Class3_'+ str(i)+'.jpg')
    y.append(3)
    dataset_without_file.append('Class4_'+ str(i)+'.jpg')
    y.append(4)

dataset = []
for i in range(len(dataset_without_file)):
    dataset.append(File(dataset_without_file[i]))

job_data_label = Job(data_label)\
                    .add_inputs(*in_files, File(csv))\
                    .add_outputs(*dataset)
        
X = dataset_without_file

X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.3, random_state=42, stratify = y_train1)

train_images = []
for i in range(len(X_train)):
    train_images.append(File(X_train[i]))
train_image_file1 = File("Training_images_Preprocess1.npy")
train_label_file1 = File("Training_labels_Preprocess1.npy")
job_training_preprocess1 = Job(training_preprocess1)\
                    .add_inputs(*train_images)\
                    .add_outputs(train_image_file1,train_label_file1)

train_image_file2 = File("Training_images_Preprocess2.npy")
train_label_file2 = File("Training_labels_Preprocess2.npy")
job_training_preprocess2 = Job(training_preprocess2)\
                    .add_inputs(train_image_file1,train_label_file1)\
                    .add_outputs(train_image_file2,train_label_file2)
val_images = []
for i in range(len(X_val)):
    val_images.append(File(X_val[i]))
val_image_file = File("Val_images_Preprocess.npy")
val_label_file = File("Val_labels_Preprocess.npy")
job_val_preprocess = Job(val_preprocess)\
                    .add_inputs(*val_images)\
                    .add_outputs(val_image_file,val_label_file)

test_images = []
for i in range(len(X_test)):
    test_images.append(File(X_test[i]))
test_image_file = File("Test_images_Preprocess.npy")
test_label_file = File("Test_labels_Preprocess.npy")
job_test_preprocess = Job(test_preprocess)\
                    .add_inputs(*test_images)\
                    .add_outputs(test_image_file,test_label_file)

wf.add_jobs(job_data_label,job_training_preprocess1,job_training_preprocess2,job_val_preprocess,job_test_preprocess)                                    

In [ ]:
len(X_test),len(X_)

In [8]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2021.03.09 21:07:17.901 UTC:
2021.03.09 21:07:17.907 UTC:   -----------------------------------------------------------------------
2021.03.09 21:07:17.913 UTC:   File for submitting this DAG to HTCondor           : Galaxy-0.dag.condor.sub
2021.03.09 21:07:17.919 UTC:   Log of DAGMan debugging messages                 : Galaxy-0.dag.dagman.out
2021.03.09 21:07:17.924 UTC:   Log of HTCondor libra

[##################################################] 100.0% ..Success (Completed: 21, Queued: 0, Running: 0, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/Galaxy2/scitech/pegasus/Galaxy/run0002
Total jobs         :     21 (100.00%)
# jobs succeeded   :     21 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#    